# LangGraph practice#1 구성도 

![구성도]()


In [1]:
import os,sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('utils'), '..')))
from module.utils import * 
from module.prompt import * 

In [3]:
strt_langsmith('practive_2')

LangSmith 추적을 시작합니다.
[프로젝트명]
practive_2


In [ ]:
loader = get_pdf_loader()
splitter = get_text_splitter()
docs = get_docs(loader,splitter)
embedding = get_embedding()
retriever = get_retriever(docs,embedding)
retriever_tool = get_retriever_tool(retriever)

In [6]:
from typing import TypedDict, Annotated, List, Literal
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    question : Annotated[list,add_messages] 
    answer : Annotated[str,"Answer"]
    is_relevant : Annotated[str,"yes or no"]
    messages : Annotated[list,add_messages]
    document : Annotated[list,'grade_document']


In [7]:
def routing(state : State)->Literal['retriever','web_search',END]:
    """ 
        문서와 연관성이 있는지 확인, 연관성 없는경우 웹검색 수행
    """
    query = state['question'][-1].content
    messages = 'RAG'
    if messages == 'RAG':
        return 'retriever'
    elif messages == 'WEB':
        return 'web_search'
    else:
        return END
    
def web_search(state : State)->State:
    """
        tavily search 활용
    """
    query = state['question'][-1].content
    messages = ''
    return State({'messages':messages})

def retriever(state:State) -> State :
    """ 
        retriever 에서 데이터 검색 수행 
    """
    query = state['question'][-1].content
    messages = ''
    return State({'messages':messages})

def grade_document(state : State) -> State :
    """ 
        검색된 데이터 별로 관련성 평가를 통해 문서를 걸러내는 작업
    """
    query = state['question']
    document = ''
    return State({'document':document})

def is_grade_decision(state : State)->Literal["re_write_query","generate"]:
    """ 
        grade_document size가 0개 경우 query 변경하여 routing 부터 재수행
    """
    if len(state['document'])  == 0 :
        return 're_write_query'
    return  'generate'

def re_write_query(state : State) -> State :
    """ 
        question  을 새로 작성하여 question 에 저장하고 routing 재수행
    """
    query = state['question'][-1].content
    re_query = ''
    return State({'question':re_query})


def generate(state : State)-> State:
    """
        web_search or relevant에서 제공해준 데이터를 기반으로 llm context 증강하여 데이터 생성
    """

    query = state['question'][-1].content
    context = state['document']
    answer = '' 
    return State({'answer':answer})

def is_hallucination(state : State ) -> Literal['is_relevant','general']:
    """
        문서 관련성 평가: 검색된 문서가 질문에 연관되었는지 여부 -> question-retrieval

        답변 관련성 평가: 생성된 답변이 질문을 잘 해결하는지 여부 -> question-answer

        답변 사실 근거 평가: 생성된 답변이 사실에 근거하는지 여부 -> retrieval-answer
        'retrieval-answer' 평가
    """
    result = 'yes'
    if result == 'yes':
        return 'is_relevant'
    else:
        return 'general'

def is_relevant(state : State ) -> Literal['re_write_query',END]:
    """
        문서 관련성 평가: 검색된 문서가 질문에 연관되었는지 여부 -> question-retrieval

        답변 관련성 평가: 생성된 답변이 질문을 잘 해결하는지 여부 -> question-answer

        답변 사실 근거 평가: 생성된 답변이 사실에 근거하는지 여부 -> retrieval-answer

        'question-answer' 평가
    """
    result = 'yes'
    if result == 'yes':
        return 'is_relevant'
    else:
        return 'general'

In [ ]:
state_graph = StateGraph(State)
